In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [15]:
# URL of the website
URL = "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile"

# Set up the Chrome driver
driver = webdriver.Chrome()
driver.get(URL)
driver.maximize_window()
time.sleep(5)  # Wait for the page to load

In [16]:
# Function to scrape bus routes
def scrape_bus_routes():
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Scrape the first page
all_bus_routes_link, all_bus_routes_name = scrape_bus_routes()

# Function to scrape bus details
def scrape_bus_details(url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Scroll down to load all bus items
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Find bus item details
        bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type= driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time= driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration= driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating= driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price= driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

        bus_details = []
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name[i].text,
                "Bus_Type": bus_type[i].text,
                "Departing_Time": departing_time[i].text,
                "Duration": duration[i].text,
                "Reaching_Time": reaching_time[i].text,
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text,
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Iterate over each bus route link and scrape the details
for link, name in zip(all_bus_routes_link, all_bus_routes_name):
    bus_details = scrape_bus_details(link, name)
    if bus_details:
        all_bus_details.extend(bus_details)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('wb2_bus_details.csv', index=False)

# Close the driver
driver.quit()

In [17]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...,Express Line,Volvo 9600 Multi Axle Semi-Sleeper (2+2),07:30,04h 25m,11:55,4.6,494,34 Seats available
1,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),08:15,04h 30m,12:45,4.5,INR 400,21 Seats available
2,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...,Express Line,Volvo A/C Seater (2+2),23:45,05h 00m,04:45,4.5,494,37 Seats available
3,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...,Shyamoli Paribahan Pvt Ltd,VE A/C Seater (2+2),23:30,05h 00m,04:30,4.4,495,24 Seats available
4,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...,Greenline,Volvo A/C Seater (2+2),06:30,05h 10m,11:40,4.5,INR 400,17 Seats available
...,...,...,...,...,...,...,...,...,...,...
170,Kolkata to Mandarmani,https://www.redbus.in/bus-tickets/kolkata-to-m...,Express Line,Volvo 9600 Multi Axle Semi-Sleeper (2+2),07:30,03h 40m,11:10,4.5,494,34 Seats available
171,Kolkata to Mandarmani,https://www.redbus.in/bus-tickets/kolkata-to-m...,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),08:15,03h 45m,12:00,4.4,INR 400,21 Seats available
172,Kolkata to Mandarmani,https://www.redbus.in/bus-tickets/kolkata-to-m...,Shyamoli Paribahan Pvt Ltd,VE A/C Seater (2+2),07:15,04h 00m,11:15,4.4,360,9 Seats available
173,Kolkata to Mandarmani,https://www.redbus.in/bus-tickets/kolkata-to-m...,AdSagufta Travels (Mayuri),A/C Seater (2+2),23:15,04h 30m,03:45,3.3,INR 365,20 Seats available
